In [27]:
import torch
from botorch.models import SingleTaskGP, ModelListGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.fit import fit_gpytorch_model



In [22]:
import DTLZ2_model
import fischer_model
from botorch.test_functions.multi_objective import DTLZ2
from DTLZ2_model import neg_l1_dist
from DTLZ2_model import predict_DTLZ2_model
from fischer_model import predict_fischer_model
import torch
import numpy as np
import pandas as pd

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
dim = 4
N = 100
#bounds of the inputs
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)]).to(device=device, dtype=dtype)


In [24]:
train_x = torch.rand(100, 4, dtype=torch.float32)
train_x = draw_sobol_samples(bounds=bounds, n=1, q=N).squeeze(0).to(device=device, dtype=dtype)
train_y = predict_fischer_model(train_x)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step


In [25]:
train_x.dtype

torch.float32

In [28]:
# Initialize models for each objective
models = [SingleTaskGP(train_X=train_x, train_Y=train_y[:, i].unsqueeze(-1)) for i in range(train_y.shape[-1])]

# Fit models
mlls = [ExactMarginalLogLikelihood(model.likelihood, model) for model in models]
for mll in mlls:
    fit_gpytorch_model(mll)

# Combine models into a ModelListGP
model = ModelListGP(*models)

In [17]:
gp = ModelListGP(train_x, train_y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

TypeError: torch.DoubleTensor is not a Module subclass

In [9]:
predict_fischer_model(torch.tensor([[0,0,0,0],[1,1,1,1]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


tensor([[-0.0089, -0.0032, -0.0340,  0.0510],
        [ 1.0114,  0.6644,  0.9430,  0.6162]])

In [2]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [3]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

In [4]:
candidate, acq_value 

(tensor([[0.5084, 0.5110]]), tensor(0.9981, dtype=torch.float64))